In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from typing import Union, List
import joblib
import imblearn
import warnings
from collections import Counter
from imblearn.over_sampling import SMOTE

#warnings.filterwarnings(action='ignore')

## load data

In [12]:
df = pd.read_csv('modified_0420.csv')

In [3]:
#df = p.get_df()
print(df.shape)
df = df.drop(columns=['Unnamed: 0'])
df

(1131682, 91)

In [19]:
scaler = MinMaxScaler()
scaler.fit(df)
scaled_df = pd.DataFrame(scaler.transform(df), index = df.index, columns = df.columns)

In [20]:
scaled_df

,loan_amnt,term,int_rate,installment,sub_grade,emp_length,loan_status,delinq_2yrs,inq_last_6mths,pub_rec,...,tax_liens_log,tot_cur_bal_log,tot_hi_cred_lim_log,total_acc_log,total_bal_ex_mort_log,total_bal_il_log,total_bc_limit_log,total_cu_tl_log,total_il_high_credit_limit_log,total_rev_hi_lim_log
0,0.139241,0.0,0.103583,0.105319,0.117647,0.2,0.0,0.02381,0.000,0.000000,...,0.0,0.697662,0.699356,0.622291,0.731735,0.757636,0.691673,0.000000,0.751610,0.620630
1,0.574684,1.0,0.766355,0.393180,0.676471,1.0,1.0,0.02381,0.125,0.000000,...,0.0,0.858478,0.830503,0.746129,0.851564,0.882764,0.713436,0.316359,0.864292,0.640116
2,0.392405,0.0,0.068536,0.284391,0.029412,1.0,1.0,0.00000,0.000,0.000000,...,0.0,0.742873,0.756081,0.561947,0.702638,0.722877,0.790299,0.000000,0.742288,0.692246
3,0.101266,0.0,0.198988,0.080883,0.205882,0.5,0.0,0.00000,0.000,0.023256,...,0.0,0.785201,0.760858,0.298221,0.619883,0.557767,0.657291,0.000000,0.645007,0.591416
4,0.493671,0.0,0.182243,0.372405,0.205882,1.0,0.0,0.00000,0.000,0.000000,...,0.0,0.762826,0.753043,0.647718,0.750725,0.000000,0.669366,0.000000,0.769280,0.634642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131677,0.658228,1.0,0.465732,0.382451,0.411765,0.2,0.0,0.00000,0.000,0.000000,...,0.0,0.796052,0.773154,0.429656,0.705611,0.000000,0.708296,0.000000,0.668224,0.651928
1131678,0.078481,0.0,0.571651,0.073688,0.588235,0.7,1.0,0.00000,0.000,0.000000,...,0.0,0.721832,0.705950,0.298221,0.635009,0.000000,0.631528,0.000000,0.652630,0.575632
1131679,0.468354,0.0,0.053349,0.336569,0.058824,0.4,0.0,0.00000,0.000,0.000000,...,0.0,0.798741,0.778508,0.601347,0.732846,0.000000,0.703171,0.000000,0.733715,0.634418
1131680,0.230380,0.0,0.351246,0.188083,0.294118,0.9,0.0,0.00000,0.125,0.000000,...,0.0,0.766240,0.739972,0.272123,0.643310,0.000000,0.600121,0.000000,0.635443,0.551056


## Design Matrix

In [22]:
df_X = scaled_df.drop(columns=['loan_status'])
df_y = scaled_df['loan_status']

pca1 = PCA(n_components = 56)
pca1.fit(df_X)
df_Xpca = pca1.transform(df_X)

Xpca_train, Xpca_test, ypca_train, ypca_test = train_test_split(df_Xpca, df_y, test_size = 0.2, random_state = 30)

In [23]:
df_X = scaled_df.drop(columns=['loan_status'])
df_y = scaled_df['loan_status']

In [24]:
pca2 = PCA(n_components = 86)
pca2.fit(df_X)
df_Xpca = pca2.transform(df_X)

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size = 0.2, random_state = 30)

## Fitting

In [31]:
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

## RidgeClassifier

In [27]:
clfRidge = RidgeClassifierCV().fit(X_train, y_train)

In [28]:
y_pred = clfRidge.predict(X_test)
joblib.dump(clfRidge, 'clfRidge_1.pkl')
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.80      0.99      0.89    180654
         1.0       0.58      0.04      0.07     45683

    accuracy                           0.80    226337
   macro avg       0.69      0.52      0.48    226337
weighted avg       0.76      0.80      0.72    226337



In [29]:
pcaRidge = RidgeClassifierCV().fit(Xpca_train, ypca_train)

In [30]:
y_pred = pcaRidge.predict(Xpca_test)
joblib.dump(pcaRidge, 'pcaRidge_1.pkl')
print(classification_report(ypca_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.80      0.99      0.89    180654
         1.0       0.58      0.03      0.06     45683

    accuracy                           0.80    226337
   macro avg       0.69      0.51      0.48    226337
weighted avg       0.76      0.80      0.72    226337



## LassoClassifier

## LDA

In [ ]:
    def scaling_data(
            self,
            scaler:MinMaxScaler|StandardScaler=StandardScaler,
            threshold:float=0.95,
            over_sampler:SMOTE=None, #RandomOverSampler
            under_sampler:TomekLinks=None)->None:
        x_scaled = scaler().fit_transform(self.x_train)
        self.input_dim = self.__get_n_components_from_pca(x_scaled, threshold=threshold)
        self.model_name+=('scaler:'+scaler.__name__+'_pca:'+str(int(threshold*100))+'_input:'+str(self.input_dim))
        self.x_train = PCA(n_components=self.input_dim).fit_transform(x_scaled).astype('float32')
        if over_sampler!=None:
            os = over_sampler(random_state=30)
            self.x_train, self.y_train = os.fit_resample(self.x_train,self.y_train)
            self.model_name+='_sampler:'+os.__class__.__name__
            print(f"====Data set resampled(oversampled)_{os.__class__.__name__}")
        elif under_sampler!=None:
            us = under_sampler(random_state=30, sampling_strategy='majority')
            self.x_train, self.y_train = us.fit_resample(self.x_train, self.y_train)
            self.model_name+='_sampler:'+us.__class__.__name__
            print(f"====Data set resampled(undersampled)_{us.__class__.__name__}")
        print(f"==label ratio")
        print(f"True Label:\t{np.sum(self.y_train==1)/len(self.y_train)}")
        print(f"False Label:\t{np.sum(self.y_train==0)/len(self.y_train)}")
        self.x_train, self.x_validation, self.y_train, self.y_validation = train_test_split(self.x_train, self.y_train, test_size=0.2, stratify=self.y_train, random_state=30)
